개인 공부 목적으로 BERT모델을 이용한 Baseline 코드에 주석을 한 줄씩 달아봤습니다.

reference : https://www.kaggle.com/code/suraj520/beginner-friendly-bert (이코드 원본)
reference : https://www.kaggle.com/code/kimseunghee/kr-code-comments-beginner-friendly-bert (한글 주석 달아주신분)

For my own learning, I've provided Korean comments on each line of the baseline code

<div style="background-color:#5D73F2; color:#19180F; font-size:40px; font-family:Arial; padding:10px; border: 5px solid #19180F; border-radius:10px"> Beginner friendly approach </div>
<div style="background-color:#D5D9F2; color:#19180F; font-size:15px; font-family:Arial; padding:10px; border: 5px solid #19180F; border-radius:10px">
<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
BERT based approach. Know more about the architecture of BERT via <a href="https://www.kaggle.com/code/suraj520/bert-know-fit-infer"> kernel </a>   </div>
</div>


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Importing modules
    </div>

In [1]:
# 모듈 임포트
import torch # Pytorch 딥러닝을 위한 오픈 소스 프레임 워크, 텐서 연산, 신경망 모델, 최적화 알고리즘 등을 제공
import torch.nn as nn # torch에서 제공하는 싱경망 모듈, 다양한 종류의 신경망 모델을 구축할 수 있다.
from transformers import BertModel, BertTokenizer # transformer 라입러리에서 BERT, Tokenizer 모델을 가져와서 사용할 수 있게 해줌
import pandas as pd # 데이터 분석 라이브러리, 테이블 형식의 데이터를 다루는 데 유용
from sklearn.model_selection import train_test_split # 데이터셋을 학습 세트와 테스트 세트로 분할하기 위한 라이브러리

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Loading the data    </div>

In [2]:
# 데이터 로드
# 해당 경로에 있는 데이터셋을 읽어와 train_data, test_data에 저장
train_data = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
test_data = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')

In [3]:
display(train_data.head())
display(test_data.head())

,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757


,student_id,prompt_id,text
0,000000ffffff,abc123,Example text 1
1,111111eeeeee,def789,Example text 2
2,222222cccccc,abc123,Example text 3
3,333333dddddd,def789,Example text 4



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Preprocessing the data (데이터 전처리)    </div>

In [4]:
'''
BertTokenizer는 Hugging Face의 transformers 라이브러리에서 제공하는 BERT 모델을 위한 토크나이저로, 텍스트 모델이 처리할 수 있는 형태의 토큰으로 변환한다.
from_pretrained('모델 이름 또는 경로') : 특정 모델을 인자로 제공하면 해당 모델의 토크나이저 config 및 vocab을 로드할 수 있다.
'''
tokenizer = BertTokenizer.from_pretrained('/kaggle/input/hugging-face-models-safe-tensors/bert-base-uncased')



'''
# 'BertTokenizer' 를 사용하여 텍스트 데이터를 BERT 모델에 입력 가능한 형태로 변환하는 작업을 수행

- batch_encode_plus는 여러개의 텍스트 문장을 한 번에 인코딩 하는데 사용된다.
- tolist()는 pandas의 DataFrame이 제공하는 메서드로, 해당(df['text'] column) 데이터를 리스트로 변환한다.
- tolist()를 하는 이유 : batch_encode_plus는 입력으로 Python 리스트, 튜플 또는 다른 시퀀스 형태의 데이터를 받을 수 있기때문에
- turncation = True : 텍스트의 길이가 모델의 max_length를 초과할 경우, 텍스트를 잘라낸다. (모델의 최대 길이 : 512)
- padding = True : 인코딩된 텍스트가 동일한 길이를 갖도록 패딩을 추가.
'''
train_encodings = tokenizer.batch_encode_plus(
    train_data['text'].tolist(),
    truncation = True,
    padding = True
)


# test data에도 동일하게 적용
test_encodings = tokenizer.batch_encode_plus(
    test_data['text'].tolist(),
    truncation = True,
    padding = True
)



In [21]:
# type : transformers.tokenization_utils_base.BatchEncoding
# input_ids, token_type_ids, attention_mask
test_encodings['token_type_ids']

[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]

In [ ]:
'''
{'input_ids': [[101, 2742, 3793, 1015, 102], [101, 2742, 3793, 1016, 102], [101, 2742, 3793, 1017, 102], [101, 2742, 3793, 1018, 102]], 
'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 
'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}
'''

In [22]:



# Tensor 개념 참고링크 : https://rekt77.tistory.com/102
# Tensor는 배열의 집합, matrix의 집합이 Tensor기 때문에 3차원부터 시작한다.

'''
- torch.utils.data.TensorDataset : 텐서들의 데이터셋을 만든다. 여러개의 텐서 데이터를 하나의 데이터셋으로 묶을 때 사용
- 위에서 tokenizer.batch_encode_plus()를 사용해서 인코딩한 결과에서 'input_ids'를 가져온다.
- 'input_ids'는 토큰화된 텍스트를 나타내는 숫자 ID의 리스트이다.
- 'attention_mask' 는 0,1 로 구성되어있는데, 실제 토큰은 1, 패딩 토큰은 0의 값을 가진다.
- .tolist() : torch.tensor가 pandas의 Series객체를 직접 처리할 수 없어서 리스트로 변환 ('content, wording')
   torch.tensor() numpy 배열, python 리스트 등과 같은 데이터 형식을 받을 수 있다.
'''

train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),   # BERT 모델의 입력으로 사용될 'input_ids'를 Torch 텐서로 변환하여 전달
    torch.tensor(train_encodings['attention_mask']), # BERT 모델의 입력으로 사용될 'attention_mask'를 Torch 텐서로 변환하여 전달
    torch.tensor(train_data['content'].tolist()), # 이 텐서는 학습과정에서 예측값과 실제값을 비교하는데 사용될 예정
    torch.tensor(train_data['wording'].tolist())
)


# test data에도 동일하게 적용
test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask'])
)



In [25]:
# train_dataset 내용 확인

print("첫 번째 문장의 Inputs ids", train_dataset[0][0]) # 토큰화된 텍스트를 나타내는 숫자 ID의 리스트, 값이 0으로 들어가있는거는 패딩으로 채워짐
print("첫 번째 문장의 attention Mask", train_dataset[0][1]) # 0, 1로 이루어졌고, 실제 토큰은 1, 패딩 토큰은 0의 값을 가짐
print("첫 번째 문장의 'content'", train_dataset[0][2]) # content 열의 첫번째 행에 있는 데이터 값
print("첫 번째 문장의 'wording'", train_dataset[0][3]) # wording 열의 첫번째 행에 있는 데이터 값

print("첫 번째 문장의 Inputs ids shape ", train_dataset[0][0].shape) 
print("첫 번째 문장의 attention Mask shape", train_dataset[0][1].shape)

첫 번째 문장의 Inputs ids tensor([  101,  1996,  2353,  4400,  2001,  2019,  7551,  3406,  2156,  2129,
         2111, 14831,  2000,  1037,  2047,  2028,  3003,  2231,  1012,  2009,
         4227,  6217,  2004,  2111,  2359,  2000,  3046,  2047,  2477,  1012,
         1996,  2493,  3582,  2505,  2008,  2003,  2056,  1998,  2707,  3810,
         2006,  2169, 14573,  2121,  2000,  5114,  3020,  2373,  1012,  2027,
         2018,  2000,  2644,  1996,  4654,  4842, 13665,  2004,  2205,  2116,
         2111,  2288,  2000,  7490,  2007,  2009, 25734,  2206,  2045,  3003,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,  


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Defining the BERT model (BERT)모델 정의하기   </div>

In [ ]:
## nn.Module 모든 모델의 근간이 되는 기본 클래스

'''
nn.Module은 PyTorch의 신경망 레이어 및 모델을 구성하는 데 필요한 핵심 구성 요소를 제공 
nn 라이브러리는 Neural Network의 모든 것을 포괄하는 모든 신경망 모델의 Base Class이다. 즉 모든 신경망 모델은 nn.Module의 subclass라고 할 수 있다.


1. 모듈화 (Modularity): nn.Module을 사용하면 모델을 여러 개의 모듈로 나눌 수 있습니다. 이렇게 모델을 모듈로 구성하면 모듈 간의 재사용성이 높아지고
   복잡한 모델을 쉽게 구성할 수 있습니다. 예를 들어, 컨볼루션 레이어, 순환 레이어, 선형 레이어 등의 서로 다른 모듈을 조합하여 모델을 만들 수 있습니다.

2. 매개변수 관리: nn.Module은 모델 내의 모든 매개변수 (가중치 및 편향)를 추적하고 쉽게 관리할 수 있게 합니다. 
   이를 통해 모델의 가중치 초기화, 매개변수 업데이트, 저장 및 로드 등을 편리하게 수행할 수 있습니다.

3. 순전파 정의: 모델의 순전파 연산을 forward 메서드에 정의합니다. 이 메서드는 입력 데이터를 받아 모델을 통과시켜 출력을 반환하는 역할을 합니다. 
   forward 메서드를 정의함으로써 모델이 어떻게 동작하는지를 명시적으로 표현할 수 있습니다.

4. .to() 메서드: nn.Module 클래스를 상속한 모든 모듈은 .to(device) 메서드를 사용하여 CPU 또는 GPU로 이동할 수 있습니다.
    이를 통해 하드웨어 가속을 활용하여 모델을 학습 및 추론할 수 있습니다.

5. 모델 저장 및 로드: nn.Module 클래스는 모델의 상태를 저장하고 로드하는 데 사용되는 메서드를 제공합니다. 이를 통해 학습된 모델을 저장하고 나중에 재사용할 수 있습니다.


__init__() : 모델을 만들때 그 구조를 정의 해줘야 하는데 __init__()을 통해 만드는게 규칙
forward() : 모델이 어떻게 forward propagation을 할지 정의하는 부분, 똑같이 레이어를 만들더라고 어떻게 연산을 할 것인지에 따라 완전히 다른 모델이 된다.

위 두가지만 있어도 모델은 정의되고 forward/backward를 문제 없이 수행하고 학습 할 수 있음.


'''



class BERTModel(nn.Module): # Pytorch의 nn.Module 클래스를 상속받아서 사용한다.
    def __init__(self): # 모델의 구조와 동작을 정의하는 생성자를 정의. 이는 파이썬에서 객체가 갖는 속성값을 초기화하는 역할로, 객체가 생성될 때 자동으호 호출
        super(BERTModel, self).__init__() # 부모 클래스인 nn.Module의 초기화 메서드를 호출
        
        # pretrain 된 BERT 모델 로드 bert-base-uncased"는 소문자만 고려하는 BERT의 기본 버전
        # 12개의 transformer 계층으로 구성되어 있으며, 각 토큰에 대해 768차원의 벡터를 출력한다.
        self.bert = BertModel.from_pretrained('/kaggle/input/hugging-face-models-safe-tensors/bert-base-uncased') 
        
        
        
        # dropout 계층으로 신경망 학습 중 과적합을 방지하기 위한 목적으로 사용, dropout 계층은 학습 과정에서 일부 뉴런을 임의로 비활성화 
        # (즉,그 출력을 0으로 설정)하여 특정 뉴런에 너무 의존하지 않도록 설정
        self.dropout = nn.Dropout(0.1) # overfitting 방지를 위한 드롭아웃
        
        
        # self.linear1, self.linear2 이들은 선형(또는 완전 연결)계층 선형 계층은 입력 벡터와 가중치 행렬 사이의 선형 변환(linear transformation)을 수행.
        # BERT가 생성한 임베딩 벡터들에 대해 추가적인 변환을 수행한다.
        # nn.Linear(768, 256)과 같은 크기 설정은 BERT 모델의 출력 차원과 관련 ,bert-base-uncased 모델은 각 입력 토큰에 대해
        # 768차원 벡터를 출력하므로 첫 번째 선형 계층은 입력 차원도 768로 설정 되어 있어야 함.
        # 256은 첫번째 선형 계층(self.linear1)에서 생성할 새로운 feature vector의 크기를 정하는 값
        self.linear1 = nn.Linear(768, 256)# 첫 번째 선형 레이어 입력 크기는 768(BERT 모델의 히든 레이어 크기), 출력 크기는 256
        
        
        
        # nn.Linear(256, 2)에서 2는 최종 출력 클래스 수를 나타내며 문제 종류나 데이터 셋이 따라 달라진다. 
        # 예를 들어 binary classification 문제인 경우 '2'가 된다.
        self.linear2 = nn.Linear(256, 2) # 두 번째 선형 레이어. 입력크기는 256 출력크기는 2
        
        '''
        레이어 크기가 모델의 어떤 영향을 끼치나?
        레이어의 크기는 신경망 모델의 성능과 복잡도에 중요한 영향을 준다.
        
        모델의 용량(Capacity) : 레이어의 크기가 클수록, 모델은 더 많은 정보를 학습하고 복잡한 패턴을 인식할 수 있고 이는 모델의 "용량"을 증가 시킨다.
        반면에 너무 작은 레이어 크기는 모델이 충분히 학습하지 못하게 만들 수 있다(underfitting) 너무 크면(overfitting)
        
        계산 비용(Computational Cost) : 더 큰 레이어를 가진 모델은 일반적으로 계산 비용이 더 많이 들고 이는 훈련 시간과 메모리 사용량 증가로 이어진다.
        
        모델의 구조(Model Archi'tecture) : 일부 경우에서, 입력 데이터나 문제 자체가 특정 차원의 출력을 요구 할 수 있다. ( e.g. 분류 문제에서 클래스 수 )
        이런 경우  해당 차원으로 출력하는 계층(layer)를 마지막에 추가

        '''
    
    
    
    
    
    
    
    
    
    def forward(self, input_ids, attention_mask):
        
        # input_ids와 attention_mask를 BERT 모델에 전달하고 BERT 모델은 입력 텍스트에 대한 정보를 잘 인코딩 하여 출력하고 그 결과를 outputs에 담는다.
        outputs = self.bert(input_ids= input_ids, attention_mask= attention_mask)  
        
        
        pooled_output = outputs.pooler_output # pooled_output은 문장에 대한 고수준 정보를 담고 있는 벡터로, 일반적으로 문장 분류 작업에 사용
        pooled_output = self.dropout(pooled_output) # Dropout 계층을 통과하여  overfitting을 방지하도록 한다.
        '''
        pooler_output이란? BERT의 마지막 Transformer 계층에서 [CLS] 토큰의 출력을 가져와서
        추가적인 Dense 레이어를 통과시킨 결과이다. 
        이 CLS 토큰의 임베딩은 전체 입력 시퀀스의 문맥적 요약으로 샤용된다.
        따라서 이 CLS 토큰의 임베딩(pooler_output)을 이용해서 추가 작업을 한다.
        '''
        
        output = self.linear1(pooled_output) # 첫 번째 선형 레이어 통과
        output = nn.ReLU()(output) # activation function인 ReLU 적용하여 비선형성(non-linearity)을 추가
        output = self.linear2(output) # 두 번째 선형 레이어 통과시키고 결과 반환
        return output
    
    """
    이런 식으로 구성된 이유는, 사전학습된 BERT 모델이 이미 복잡한 자연어 처리 작업에 필요한 대부분의 정보를 인코딩할 수 있기 때문입니다. 
    그러나 우리가 해결하려는 구체적인 문제(예: 분류 문제 등)에 따라 추가적인 변환 및 학습이 필요할 수 있습니다.
    따라서 이 코드에서는 BERT 모델의 출력인 'pooler_output' 벡터를 추가적인 선형 계층들(self.linear1, self.linear2)로 보내어 
    원하는 형태(여기서는 2개 클래스에 대한 확률 분포일 가능성이 높습니다.)로 변환하도록 하였습니다."""


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Training the BERT model    </div>

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # CUDA가 사용 가능하면 device는 'cuda'로 설정되고, 아니면 'cpu'로 된다.

# 앞에서 정의한 BERTModel 클래스의 인스턴스인 model을 생성하고 to(device)를 앞서 정의한 장치(device)로 보낸 후 계산 진행
model = BERTModel().to(device) 


'''
신경망 학습의 목표는 주어진 데이터에 대해 잘 예측하는 모델을 만드는 것이기 때문에 이를 위해서는 모델의 파라미터(가중치와 편향 등) 적절하게 조정해야 한다. 
이 파라미터 조정 과정이 최적화(Optimization)이다.  최적화 과정에서는 손실 함수(Loss Function)를 최소화 하는 파라미터 값을 찾는다.
손실 함수 값은 모델의 예측과 실제 값 사이의 차이를 나타내므로 이 값을 최소화 한다는 것은 모델의 예측 성능을 최대화 한다는 것과 동일.
옵티마이저(Optimizer)는 이러한 최적화 과정을 수행하는 알고리즘이고 여기서 사용된 AdamW 옵티마이저 같은 경우 , 각 파라미터에 대한 손실 함수의 
그래디언트(Gradient, 미분값)를 계산하고, 그 정보를 사용하여 파라미터 값을 업데이트. 따라서 옵티마이저를 설정하는 것은 신경망 학습에 필수적인 부분으로
어떻게 가장 좋은 성능을 내는 모델 파라미터 값을 찾아낼지 경정하는 방법을 정의하는거다.
'''


# 최적화 알고리즘으로 AdamW를 사용, Adam(Adaptive momentum, momentum이 도입된 optimizer)  최적화 방법에 가증치 감쇠(weight deecay)를 추가한 것
# model.parameters()는 최적화할 파라미터들(모델의 가중치 등)이며 lr=le-5 는 학습률(learning rate)이다.
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5) # optimizer 정의 model.parameters()는 자동으로 모델 내의 모든 파라미터를 가져온다.
criterion = nn.MSELoss() #loss function으로 meab Squared Error 사용, 예측 값과 실제 값 사이의 차이를 측정하는데 사용


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Creating data loader and performing sanity check (데이터 로딩 및 기본적인 검증 작업을 수행하는 과정)  </div>

In [ ]:
batch_size = 16

In [ ]:
# splitting training data into train and validation sets (훈련 데이터를 훈련 및 검증 데이터로 분한하기)
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2, random_state=0)
# 데이터를 train용 데이터와 validation 데이터로 나눈다. train_dataset은 80%로, val_dataset은 20%로

# Creating train loader
# 학습 데이터를 로딩하기 위한 데이터로더 생성. 
# train_dataset : 학습 데이터셋을 로드하고 사용, 모델을 학습시킬 때 사용되는 데이터
# batch_size : 한번의 배치(batch_size)에 포함될 데이터 샘플의 수를 지정, 배치는 모델의 가중치를 업데이트할 때 사용
# shuffle=True : 데이터를 섞을지 여부를 지정 'True'로 설정하면 데이터가 에폭(epoch)마다 섞여 모델의 훈련을 안정화시키는데 도움을 줌, 즉 학습 데이터가 에폭마다 순서가 바뀜
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


# Creating validation loader
# 검증 데이터를 로딩하기 위한 데이터로더 생성. 
# val_dataset : 검증 데이터셋을 로드하고 사용, 모델을 성능을 평가하는데 사용
# batch_size : 마찬가지로 검증 데이터도 배치단위로 처리
# shuffle=True : 검증 데이터는 보통 순서대로 사용하며 섞지 않는것이 일반적
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

'''
torch.utils.data.DataLoader는 Pytorch에서 제공하는 DataLoader
데이터 로더를 이용하면 데이터를 쉽게 배치로 분할하여 모델 train이나 validataion에 사용할 수 있다.
일반적으로 train data는 shuffle을 하고, validattion data는 shuffle을 안 한다,
- 이유 
train data는 shuffle 하는 이유는 배치 학습의 안정성 향상을 위해서다. 학습 데이터를 그대로 사용하면 순서대로 모델을 훈련하게 되는데
이 경우 모델은 데이터의 순서에 의존할 수 있기때문에 모델의 학습이 불안정해질 수 있다. 데이터를 섞으면 각 에폭(epoch)마다 다른 순서로 데이터를 제공하여
모델이 다양한 데이터 패턴을 학습하기 때문에 새로운 데이터에 대한 예측 능력이 향상, 일반화(generalization) 

검증 데이터의 순서를 섞지 않는 이유는 
모델 평가의 일관성을 유지하기 위해서다. 검증 데이터는 모델의 성능을 평가하고 튜닝하는데 사용되는데 새로운 데이터에 얼마나 잘 일반화 되는지를 평가하는 것이라서
검증 데이터를 섞으면 모델의 평가가 무작위로 이루어지므로 일관된 평가가 어렵다 따라서 검증 데이터는 순서를 유지하여 모델 평가의 일관성을 유지
'''

In [ ]:
for batch in train_loader:
    print(batch)
    print(len(batch))
    break
    
'''
여기서 각각의 batch가 뜻하는 것 : (input_ids, attention_mask, content, wording)
그래서 len(batch) 찍으면 모두 4가 나온다.
'''


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Training the model for 30 epochs    </div>

In [ ]:
# Training loop
model.train() # 모델을 훈련 모드로 설정. 이렇게 하면 모델 내부에서 드롭아웃 및 배치 정규화와 같은 훈련 중에만 활성화 되어야 하는 기능들이 활성화
for epoch in range(30): # 전체 데이터셋에 대한 훈련을 30 에폭(epoch)동안 반복
    running_loss = 0.0 # 현재 에폭에서 발생한 모든 배치의 loss값을 저장하기 위한 변수
    
    # 훈련 데이터로더(train_loader)에서 배치 데이터를 로드해서 각 배치에 대해서 아래의 코드를 실행, 
    # input_ids, attention_mask, content, wording 은 데이터의 입력 특성들을 나타냄.
    for step, (input_ids, attention_mask, content, wording) in enumerate(train_loader): 
        
        # 해당 배치의 데이터들을 GPU로 이동
        input_ids = input_ids.to(device) 
        attention_mask = attention_mask.to(device)
        content = content.to(device)
        wording = wording.to(device)
        
        optimizer.zero_grad() # 이전 배치에서의 기울기 정보 초기화
        
        # 현재 배치의 data를 모델에 입력하고 예측값을 없는다. 
        # outputs는 [batch_size, 2]가 될것이다.
        outputs = model(input_ids, attention_mask) 
        
        # output loss를 계산, 모델의 예측과 실제 값(content 및 wording)간의 차이를 나타낸다.
        loss = criterion(outputs[:, 0], content) + criterion(outputs[:, 1], wording) 
        
        loss.backward() # 역전파(backpropagation)를 수행하고 (loss에 대한 gradient 계산)
        optimizer.step() # 계산된 gradient를 통해 모델 파라미터를 업데이트
        
        if step % 500 == 0: # 매 500번째 스텝(step)에서 현재 epoch step, loss 출력
            print("Epoch {}, Step {}, Loss: {}".format(epoch+1, step, loss.item()))
            
        running_loss += loss.item() # 현재 배치의 loss를 running_loss에 누적
        
    print(f"Epoch {epoch+1} Loss: {running_loss / len(train_loader)}") # 현재 에폭에서의 평균 train_los를 출력.
    
    
    # 딥 러닝 모델의 검증(validation)단계
    model.eval() # 모델을 평가 모드로 전환 이때 드롭아웃과 배치 정규화는 비활성화
    with torch.no_grad(): # 해당 컨텍스트 내에서 모든 연산이 기록되지않고 그래디언트 계산이 비활성화
        val_loss = 0.0 # validation loss을 저장하기 위한 변수, train에서와 동일하게 진행, 다만 parameter 업데이트 과정은 없음
        
        # 검증 데이터 로더(val_loader)에서 데이터를 불러온다. (validation data의 각 배치에 대해서 실행)
        for val_step, (input_ids, attention_mask, content, wording) in enumerate(val_loader): 
            
            # 데이터들을 GPU로 이동시킴
            input_ids = input_ids.to(device) 
            attention_mask = attention_mask.to(device)
            content = content.to(device)
            wording = wording.to(device)
            
            val_outputs = model(input_ids, attention_mask) # 모델에 검증 데이터를 입력하고 모델의 출력값(val_output)을 얻는다.
            
            # 검증데이터와 모델의 예측값 간의 차이를 나타내는 loss 계산하면서 계속 검증 손실을 누적한다.
            val_loss += criterion(val_outputs[:, 0], content) + criterion(val_outputs[:, 1], wording) 
            
        print(f"Validation Loss: {val_loss / len(val_loader)}") # 검증 데이터셋에 대한 평균 검증 손실을 출력
    model.train() # 다음 epoch를 위해서 훈련모드로 전환, 이렇게 하면 다음 훈련 에폭에서 다시 그래디언트 계산과 가중치 업데이트가 가능해짐


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Creating test loader    </div>

In [ ]:
# PyTorch를 사용하여 데이터 셋을 로드하고 데이터 로더를 설정하는 부분

# 데이터셋을 로드하고 데이터로더를 설정.
# test_loader 이는 데이터를 로드하고 미니배치(mini-batch)로 만들어주는 데이터 로더 객체를 나타낸다., 이를 통해 데이터 셋을 미니배치로 나누어 모델에 공급
# torch.utils.data.DataLoader : PyTorch에서 제공하는 데이터 로더 클래스를 사용
# test_dataset : 로드할 데이터 셋을 나타낸다.
# batch_size=16 : 미니배치의 크기를 설정하는 것. 데이터를 16개씩 처리하며, 한번의 역전파(backpropagation)단계를 16개의 데이터를 사용하여 가중치를 업데이트
# 메모리 사용을 효율적으로 하면서 모델 훈련을 가속화하는데도움이 됨
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Generating predictions on test set (테스트 세트에 대한 예측 생성)   </div>

In [ ]:
# 모델을 사용하여 테스트 데이터에 대한 예측값을 생성하고 predictions  리스트에 저장하는 코드

model.eval() #  모델을 평가 모드로 설정. 평가 모드에서 모델은 가중치 업데이트를 수행하지 않고 그래디언트 계산을 하지 않는다.
predictions = [] # 모델의 예측값을 저장하기 위한 리스트
with torch.no_grad(): # 그래디언트 계산을 비활성화하기 위한 블록
    for input_ids, attention_mask in test_loader: # 테스트 데이터를 배치 단위로 input_ids와 attention_mask를 불러온다.
        
        # 미니배치 내의 입력 데이터와 마스크를 GPU 보냅니다.
        input_ids = input_ids.to(device) 
        attention_mask = attention_mask.to(device)
        
        # 모델에 입력 데이터와 마스크를 전달하여 예측값을 계산. outputs에는 해당 배치의 모델 prediction 값이 저장되어 있다.
        outputs = model(input_ids, attention_mask) 
        
        # 모델의 예측값을 CPU로 이동하고, 이를 predictions 리스트에 추가, 모든 미니 배치의 예측값이 하나의 리스트에 저장.
        predictions.extend(outputs.cpu().numpy()) 
        
        '''
            outputs 텐서를 cpu()로 옮기고, numpy 배열로 변환한다.
            그 후 predictions 리스트에 이 값을 추가한다. cpu로 옯기는 이유는 (Numpy는 CPU 기반으로 동작하기때문)
            .numpy()를 이용해서 Pytorch 텐서를 numpy 배열로 변환할 때, 해당 텐서 꼭 cpu에 있어야 하기 때문 
        '''


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Generating submission
    </div>

In [ ]:
submission_df = pd.DataFrame({ # 데이터 프레임을 생성하는 부분
    'student_id': test_data['student_id'], # 첫 번째 열 'student_id'로 정의하고 test_data['student_id'] 열의 내용을 넣는다.
    'content': [pred[0] for pred in predictions], # 두 번째 열은 'content' 로 정의하고 , 이 열에는 predictions 리스트에서 각 예측값('pred')의 첫 번째 요소를 가져와 넣는다.
    'wording': [pred[1] for pred in predictions] # 세 번째 열은 'wording'으로 정의하고 이 열에는 predictions 리스트에서 각 예측값 ('pred')의 두 번째 요소를 가져와 넣는다.
})

submission_df.to_csv('submission.csv', index=False) # 생성한 데이터프레임을 CSV 파일로 저장, 행 번호는 포함되지 않도록 index=False 로 설정

In [ ]:
submission_df